## 🌀 Quick Start Guide: It's all starting to unravel!

First run `pip install unravelsports` if you haven't already!


-----


In [ ]:
%pip install unravelsports --quiet

### 1. Processing Data

1. Load [Kloppy](https://github.com/PySport/kloppy) dataset. See [in-depth Tutorial](1_kloppy_gnn_train.ipynb) on how do processes multiple match files.
2. Convert to Graph format using `GraphConverter`
3. Create dataset for easy processing with [Spektral](https://graphneural.network/) using `CustomSpektralDataset`

In [ ]:
from unravel.soccer import GraphConverter
from unravel.utils import CustomSpektralDataset

from kloppy import skillcorner

from unravel.utils import dummy_labels

# Load Kloppy dataset
kloppy_dataset = skillcorner.load_open_data(
    match_id=4039,
    include_empty_frames=False,
    limit=500,  # limit to 500 frames in this example
)

# Initialize the Graph Converter, with dataset, labels and settings
converter = GraphConverter(dataset=kloppy_dataset, labels=dummy_labels(kloppy_dataset))

# Compute the graphs and add them to the CustomSpektralDataset
dataset = CustomSpektralDataset(graph=converter.to_spektral_graphs())

### 2. Split Data

Split the dataset with the built in `split_test_train_validation` method.

In [ ]:
from spektral.data import DisjointLoader

train, test, val = dataset.split_test_train_validation(
    split_train=4, split_test=1, split_validation=1, random_seed=42
)

#### 2. Compile Model

1. Initialize the `CrystalGraphClassifier`.
2. Compile the model with a loss function, optimizer and your preferred metrics.

In [ ]:
from unravel.classifiers import CrystalGraphClassifier

from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC, BinaryAccuracy

model = CrystalGraphClassifier()

model.compile(
    loss=BinaryCrossentropy(), optimizer=Adam(), metrics=[AUC(), BinaryAccuracy()]
)

### 3. Fit Model

1. Create a [`DisjointLoader`](https://graphneural.network/loaders/#disjointloader) for training and validation sets.
2. Fit the model. Note: set `use_multiprocessing=True` to speed up training significantly. 

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

batch_size = 32
epochs = 10

loader_tr = DisjointLoader(train, batch_size=batch_size)
loader_va = DisjointLoader(val, epochs=1, shuffle=False, batch_size=batch_size)

model.fit(
    loader_tr.load(),
    epochs=epochs,
    steps_per_epoch=loader_tr.steps_per_epoch,
    use_multiprocessing=True,
    validation_data=loader_va.load(),
    callbacks=[EarlyStopping(monitor="loss", patience=5, restore_best_weights=True)],
)

### 4. Evaluate Model Performance

1. Create another `DisjointLoader`, this time for the test set.
2. Evaluate model performance on the test set. This evaluation function uses the `metrics` passed to `model.compile`

Note: Our performance is really bad because we're using random labels, very few epochs and a small dataset.

In [ ]:
loader_te = DisjointLoader(test, epochs=1, shuffle=False, batch_size=batch_size)
results = model.evaluate(loader_te.load())

### 5. Predict

1. Use unseen data to predict on. In this example we're using the test dataset.
2. We have to re-create `loader_te` because `DisjointLoader` is a generator.
3. Setting `batch_size` and `use_multiprocessing=True` on prediction helps speed up the prediction

In [ ]:
loader_te = DisjointLoader(test, batch_size=batch_size, epochs=1, shuffle=False)
loaded_pred = model.predict(loader_te.load(), use_multiprocessing=True)